<a href="https://colab.research.google.com/github/pstjohn/gnn-codecamp/blob/main/cetane_number_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

In [1]:
%tensorflow_version 2.x
import tensorflow as tf

In [2]:
tf.__version__

'2.5.0'

In [5]:
!pip install rdkit-pypi
import rdkit
from rdkit.Chem import AllChem

In [9]:
def get_morgan_fingerprint(smiles, radius=2, nBits=1024):
  mol = rdkit.Chem.MolFromSmiles(smiles)
  bv = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)

In [11]:
import numpy as np

In [13]:
np.asarray(bv).sum()

8

In [7]:
import pandas as pd

In [9]:
cn_data = pd.read_csv('https://raw.githubusercontent.com/pstjohn/gnn-codecamp/main/cn.csv')
cn_data.head()

,SMILES,IUPAC name,CN,Device
0,CCCCCCC,heptane,57.600,AFIDA
1,CCCCCCCCCCCCO,dodecan-1-ol,51.605,AFIDA
2,CCCC=O,butanal,38.315,AFIDA
3,CC1CCCCC1,methylcyclohexane,17.355,AFIDA
4,CC(C)CC(C)(C)C,"2,2,4-trimethylpentane",5.200,AFIDA


In [6]:
!pip install nfp